# Flux1.x devのサンプル利用コード
##利用方法
・基本的には全てのセルを実行することで利用できます。

・GPUを利用しましょう。（利用制限があるのでこまめにインスタンスは削除してください）

（変更する際は，「ランタイム」→「ランタイムのタイプを変更」からGPUを選択してください）

##注意点
・本ファイルの相対的な場所は変更しないでください。

・本ファイルの名前を変更しないでください。

変更する場合は，[2]セル目のファイル名も一緒に変更してください。

・[1]セル目の実行時にセッションの再起動を求められることがあります。

その場合はポップアップに従って再起動したあと，全てのセルを再度実行してください。

In [1]:
#Google Driveのフォルダをマウント（認証入る）
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#SDXL で必要なモジュールのインストール
%rm -r /content/diffusers
%cd /content/
!git clone https://github.com/huggingface/diffusers.git
!pip install -U bitsandbytes optimum-quanto peft tensorflow-metadata transformers scikit-learn ftfy accelerate invisible_watermark safetensors controlnet-aux mediapipe timm



/content
Cloning into 'diffusers'...
remote: Enumerating objects: 69299, done.
remote: Counting objects: 100% (1220/1220), done.
remote: Compressing objects: 100% (664/664), done.
remote: Total 69299 (delta 764), reused 821 (delta 441), pack-reused 68079 (from 1)
Receiving objects: 100% (69299/69299), 49.14 MiB | 23.07 MiB/s, done.
Resolving deltas: 100% (51096/51096), done.
  Using cached mediapipe-0.10.15-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.7 kB)
  Using cached timm-1.0.9-py3-none-any.whl.metadata (42 kB)
INFO: pip is looking at multiple versions of mediapipe to determine which version is compatible with other requirements. This could take a while.
  Using cached mediapipe-0.10.14-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.7 kB)
  Using cached mediapipe-0.10.13-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.7 kB)


下記のコードを実行すると，認証が求められるので，認証してください。（Google colabにマイドライブの中を読めるようにする認証です）

In [3]:
from huggingface_hub import login
from google.colab import userdata
HF_LOGIN = userdata.get('HF_LOGIN')
login(HF_LOGIN)

# カレントディレクトリを本ファイルが存在するディレクトリに変更する。
import glob
import os
pwd = os.path.dirname(glob.glob('/content/drive/MyDrive/colabzenn/colab_fluxdev_sample/FluxDev_sample.ipynb', recursive=True)[0])
print(pwd)

%cd $pwd
!pwd

import sys
sys.path.append("/content/diffusers/src")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful
/content/drive/MyDrive/colabzenn/colab_fluxdev_sample
/content/drive/MyDrive/colabzenn/colab_fluxdev_sample
/content/drive/MyDrive/colabzenn/colab_fluxdev_sample


In [4]:
#モジュールをimportする
from module.module_flux import FLUX
import time

モデルを変更する場合は，下記のconfigを変更してください。

「;」を付けるとコメントアウトできます。


In [5]:
#モデルの設定を行う。

config_text = """
[FLUX]
device = auto
n_steps=24
seed=42

;from_single_file = False
;base_model_path = black-forest-labs/FLUX.1-schnell
base_model_path = black-forest-labs/FLUX.1-dev

use_controlnet = False
controlnet_path = Shakker-Labs/FLUX.1-dev-ControlNet-Union-Pro

;control_mode0 = canny
;control_mode1 = tile
control_mode2 = depth
;control_mode3 = blur
;control_mode4 = pose
;control_mode5 = gray
;control_mode6 = lq

;lora_weight_repo0 = alfredplpl/flux.1-dev-modern-anime-lora
;lora_weight_path0 = modern-anime-lora-2.safetensors
;lora_weight_repo1 = aleksa-codes/flux-ghibsky-illustration
;lora_weight_path1 = lora.safetensors
;lora_scale0 = 1.0
;lora_scale1 = 1.0
;trigger_word0 = "modern anime style"
;trigger_word1 = "GHIBSKY style painting"

;select_solver = LCM
;select_solver = DPM
;select_solver = Eulera
select_solver = FMEuler

use_karras_sigmas = True
scheduler_algorithm_type = dpmsolver++
solver_order = 2

cfg_scale = 3.5
;cfg_scale = 1.0
width = 832
height = 1216
output_type = pil
aesthetic_score = 6
negative_aesthetic_score = 2.5

save_latent_simple = False
save_latent_overstep = False


"""

with open("configs/config.ini", "w", encoding="utf-8") as f:
  f.write(config_text)

生成する画像を決定するプロンプトを指定します。

negative_promptを設定することで，ここに書かれている内容を回避できる。


In [6]:
#読み上げるプロンプトを設定する。

# not use controlnet
main_prompt = """
modern anime style, a cute Japanese idle singer girl, long Yellowish-white hair with red small ribbon,red eyes,small red hat ,She is singing ,holding a microphone, wearing white idle costume, standing on the stage ,This is a flashy concert with 3D holograms and laser effects
"""

#use controlnet depth
#main_prompt = """
#modern anime style, a cute Japanese idle singer girl, long Yellowish-white hair with red small ribbon,red eyes,small red hat ,She sings with a smile on her face , wearing white idle costume, standing on the stage ,This is a flashy concert with 3D holograms and laser effects
#"""

#double lora and anime lora only
#main_prompt = """
#a cute Japanese idle singer girl, long Yellowish-white hair with red small ribbon,red eyes,small red hat ,She sings with a smile on her face , wearing white idle costume, standing on the stage
#"""


#use controlnet canny
#main_prompt = """
#modern anime style, a cute Japanese idle singer girl, long Yellowish-white hair with red small ribbon,red eyes
#"""

#コントロールネットに入力する参照画像は0から6までの数字を想定（0.png,6.webpなど。数字はcontrolnetのモードと一致させてください）
#参照）コントロールネットのモード　canny (0), tile (1), depth (2), blur (3), pose (4), gray (5), low quality (6).
input_refer_image_folder = "./inputs/refer"
#コントロールネットに入力する変換後の参照画像の格納フォルダ。命名規則は上記と同様
output_refer_image_folder = "./inputs/refer_prepared"





In [7]:
flux = FLUX()
#指定しているControlNetで利用できる"./inputs/refer_prepared/0.png"などがすでにある場合は、下記はコメントアウトしても良い
#flux.prepare_multi_referimage(input_refer_image_folder = input_refer_image_folder,output_refer_image_folder = output_refer_image_folder, low_threshold = 100, high_threshold = 200, noise_level=25, blur_radius=5)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

transformer quantizing
loaded transformer


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

text_encoder_2 quantizing
loaded text_encoder_2


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


loaded base model
cpu offloading


In [8]:
for i in range(5):
      start = time.time()
      image = flux.generate_image(main_prompt,image_path = output_refer_image_folder, controlnet_conditioning_scale = [0.5])
      print("generate image time: ", time.time()-start)
      image.save("./outputs/FLUX_result_{}.png".format(i))

  0%|          | 0/24 [00:00<?, ?it/s]

generate image time:  141.311452627182


  0%|          | 0/24 [00:00<?, ?it/s]

generate image time:  145.60417485237122


  0%|          | 0/24 [00:00<?, ?it/s]

generate image time:  147.65501308441162


  0%|          | 0/24 [00:00<?, ?it/s]

generate image time:  147.0672791004181


  0%|          | 0/24 [00:00<?, ?it/s]

generate image time:  147.7139115333557
